In [1]:
import praw
import os
import openai
import json
from Keys import openai_keys
openai.organization = openai_keys['organization']
openai.api_key = openai_keys['api_key']

dict_keys(['consumer_key', 'consumer_secret', 'access_token', 'access_token_secret', 'Bearer'])


In [2]:
# Load the JSON data from the file
with open('Scrappings/oppenheimer.json', 'r') as json_file:
    comments_by_post = json.load(json_file)

# # Now you can access the comments for a specific post title
# desired_post_title = "Example Post Title"
# if desired_post_title in comments_by_post:
#     comments_for_post = comments_by_post[desired_post_title]
#     print(f"Comments for '{desired_post_title}':")
#     for comment in comments_for_post:
#         print(comment)
# else:
#     print(f"No comments found for '{desired_post_title}'.")

In [ ]:
class Post:
    def __init__(self, postTitle):
        self.title = postTitle
        self.batches = list()

class CommentBatch:
    def __init__(self):
        self.comments = list()
        self.GPTAnalysis = ""
posts = list()
for sub in submissions:
    sub.comment_sort = "top"
    comments = sub.comments.list()
    c = 0
    i = 2
    post = Post(sub.title)
    while i > 0:
        prompts = [{"role": "system", "content":
              'You will be given 4 Reddit comments under a post - "' + sub.title + '", each comment is embedded into curly brackets ({}). Your goal is to write sentiment analysis of each comment towards the topic - "movie Oppenheimer", in a format RATING - KEYWORDS. Where RATING is a number from 0 to 10, where 0 is drastically negative attitude to the topic, and 10 is a drastically positive, or -1 if comment is unrelated to the topic. Where KEYWORDS are 2-3 words or phrases, separated by a comma (,), taken from the comment most indicative of reasons for such attitude. Dont write anything besides the format'}]
        prompt = ""
        j = 3
        minibatch = CommentBatch()
        while j >= 0:
            if type(comments[c]) is not praw.models.Comment:
                c+=1
                continue
            temp = "{" + comments[c].body + ("},\n" if j > 0 else "}")
            prompt += temp
            minibatch.comments.append(temp)
            j -= 1
            c += 1
        prompts.append({"role": "user", "content": prompt},)
        chat = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=prompts)
        reply = chat.choices[0].message.content
        minibatch.GPTAnalysis = reply
        post.batches.append(minibatch)
        # print(reply)
        i -= 1
    posts.append(post)